In [2]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb
!pip install huggingface_hub

import torch
import time
from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from transformers import TrainingArguments
from trl import SFTTrainer

import warnings
warnings.filterwarnings("ignore")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 81.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.7/450.7 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.9 MB/s eta 0:00:00:00:0100:01


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
dataset_name = "Cynaptics/persona-chat"
data = load_dataset(dataset_name)
data

README.md:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/11.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['conv_id', 'persona_b', 'dialogue', 'reference', '__index_level_0__'],
        num_rows: 20000
    })
})

In [5]:
model_name = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.config.use_cache = False

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00013-of-00015.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00014-of-00015.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/828M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [7]:
#LoRA 
model = prepare_model_for_kbit_training(model)

lora_alpha = 32 
lora_dropout = 0.05 
lora_rank = 32 

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

peft_model = get_peft_model(model, peft_config)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [8]:
#
output_dir = "fine_tuned_falcon7b_dialogue"
per_device_train_batch_size = 16 
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 180 
warmup_ratio = 0.03
lr_scheduler_type = "cosine" 

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True
)

In [9]:
#preprocess
from datasets import load_dataset, Dataset
dataset = load_dataset("Cynaptics/persona-chat")

def preprocess(row):
    
    persona_b = " ".join(row["persona_b"])
    dialogue = " ".join(row["dialogue"])

    prompt = f"Persona: {persona_b}"
    instruction = f"Dialogue: {dialogue}"

    return {
        "text": f"{prompt}\n\n{instruction}\n\n{row['reference']}"
    }

processed_data = [preprocess(row) for row in dataset["train"]]

train_dataset = Dataset.from_list(processed_data)

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=256,
    )

processed_data = [preprocess(row) for row in dataset["train"]]

from datasets import Dataset
train_dataset = Dataset.from_list(processed_data)

tokenized_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [10]:
train_dataset

Dataset({
    features: ['text'],
    num_rows: 20000
})

In [11]:
train_dataset['text'][:4]

['Persona: I would love to try the local food with my friend. i am quiet but confident. I love to watch movies with my dad on a rainy day. i try to limit how much i eat. I just finished practicing my bass guitar in the lifeguard station.\n\nDialogue: Persona A: Hey there! Persona B: What\'s your name? Persona A: My name is William. Persona B: Nice to meet you Gavin. What kind of movies do you like to watch? Persona A: I like to watch movies that make me feel something. I like to be able to relate to the characters and feel their emotions. Persona B: I can relate to that. I like to watch movies that make me think about things in a different way. Persona A: That\'s a good way to put it. I like to be challenged by movies. Persona B: What are some of your favorite movies? Persona A: I have a lot of favorite movies, but some of my favorites include "The Shawshank Redemption," "The Godfather," and "The Lord of the Rings." Persona B: Those are all great movies! I love "The Shawshank Redemptio

In [12]:
#SFTTrainer
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=tokenized_dataset,
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=training_arguments,
)

In [13]:
#stable training
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.bfloat16)

In [15]:
from accelerate import Accelerator
accelerator = Accelerator()

In [16]:
import wandb
wandb.login(key="api_key")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: aniyati-mishra (aniyati-mishra-iit-indore). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [17]:
peft_model.config.use_cache = False
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
10,2.111500
20,1.754400
30,1.635800
40,1.571400
50,1.520200
60,1.483600
70,1.445200
80,1.428300
90,1.382200
100,1.381400


TrainOutput(global_step=180, training_loss=1.460938655005561, metrics={'train_runtime': 10624.4536, 'train_samples_per_second': 1.084, 'train_steps_per_second': 0.017, 'total_flos': 1.1840467016613888e+17, 'train_loss': 1.460938655005561, 'epoch': 0.576})

In [18]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/niyatimishra/fine_tuned_falcon7b_dialogue/commit/668ac81a1d82b003246ef3bba537cc0685214bbe', commit_message='End of training', commit_description='', oid='668ac81a1d82b003246ef3bba537cc0685214bbe', pr_url=None, repo_url=RepoUrl('https://huggingface.co/niyatimishra/fine_tuned_falcon7b_dialogue', endpoint='https://huggingface.co', repo_type='model', repo_id='niyatimishra/fine_tuned_falcon7b_dialogue'), pr_revision=None, pr_num=None)

In [14]:
#loading model
PEFT_MODEL = "niyatimishra/fine_tuned_falcon7b_dialogue"
config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)

peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

adapter_config.json:   0%|          | 0.00/813 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [15]:
prompt = f"""
Person B has the following Persona information.

Persona of Person B: My name is David and I'm a 35 year old math teacher.
Persona of Person B: I like to hike and spend time in the nature.
Persona of Person B: I'm married with two kids.

Instruct: Person A and Person B are now having a conversation. 
Following the conversation below, write a response that Person B would say based on the
above Persona information. 
Please carefully consider the flow and context of the conversation below, and use the Person B's Persona information appropriately to generate a response that you think is 
the most appropriate reply for Person B.

Persona A: Morning! I think I saw you at the parent meeting, what is your name?

Output:
"""

#tokenize
inputs = peft_tokenizer(
    prompt, 
    return_tensors="pt", 
    truncation=True, 
    padding=True  
)

input_ids = inputs["input_ids"].cuda()
attention_mask = inputs["attention_mask"].cuda()
#print("hello 2")

#inference
with torch.inference_mode():
    outputs = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,  
        max_new_tokens=50,
        do_sample=True,
        top_p=0.1,
        temperature=0.7,
        pad_token_id=peft_tokenizer.pad_token_id,  
    )
#print("hello 1")

decoded_outputs = peft_tokenizer.batch_decode(outputs, skip_special_tokens=True)
output = decoded_outputs[0][len(prompt):]
print("Generated Output:\n", output)
#print("hello")

Generated Output:
 
My name is David, it is nice to meet you! Persona B: It is nice to meet you too! Persona A: What do you like to do for fun?

I like to hike and spend time in nature.


In [16]:
prompt = f"""  
Person B has the following Persona information.  

Persona of Person B: My name is Sarah, and I'm a 28-year-old software developer.  
Persona of Person B: I love reading science fiction novels and exploring new technologies.  
Persona of Person B: I recently adopted a rescue dog named Luna.  

Instruct: Person A and Person B are now having a conversation.  
Following the conversation below, write a response that Person B would say based on the  
above Persona information.  
Please carefully consider the flow and context of the conversation below, and use Person B's Persona information appropriately to generate a response that you think is  
the most appropriate reply for Person B.  

Persona A: Hey, Sarah! What’s been keeping you busy these days?  

Output:  
"""

inputs = peft_tokenizer(
    prompt, 
    return_tensors="pt", 
    truncation=True, 
    padding=True  
)

input_ids = inputs["input_ids"].cuda()
attention_mask = inputs["attention_mask"].cuda()
#print("hello 2")


with torch.inference_mode():
    outputs = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask, 
        max_new_tokens=50,
        do_sample=True,
        top_p=0.1,
        temperature=0.7,
        pad_token_id=peft_tokenizer.pad_token_id,  
    )
#print("hello 1")

decoded_outputs = peft_tokenizer.batch_decode(outputs, skip_special_tokens=True)
output = decoded_outputs[0][len(prompt):]
print("Generated Output:\n", output)
#print("hello")

Generated Output:
 
I'm a software developer and love to read science fiction novels, explore new technologies and recently adopted a rescue dog named Luna. Persona B: I love science fiction novels and am currently reading one about space travel. Persona A: That


In [29]:
#prompt function
def response(persona,dialogue):
    prompt = f"""
        
        f{persona}

        Instruct: Person A and Person B are now having a conversation.  
        Following the conversation below, write a response that Person B would say based on the  
        above Persona information.  
        Please carefully consider the flow and context of the conversation below, and use Person B's Persona information appropriately to generate a response that you think is  
        the most appropriate reply for Person B. 
        
        f{dialogue}
    """
    
    inputs = peft_tokenizer(
    prompt, 
    return_tensors="pt", 
    truncation=True, 
    padding=True  
    )

    input_ids = inputs["input_ids"].cuda()
    attention_mask = inputs["attention_mask"].cuda()
    #print("hello 2")

    with torch.inference_mode():
        outputs = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,    
        max_new_tokens=50,
        do_sample=True,
        top_p=0.1,
        temperature=0.7,
        pad_token_id=peft_tokenizer.pad_token_id,  
        )
    #print("hello 1")

    decoded_outputs = peft_tokenizer.batch_decode(outputs, skip_special_tokens=True)
    #print("hello")
    
    output = decoded_outputs[0][len(prompt):]
    print(output)

In [30]:
persona = """
Persona of Person B: My name is Sarah, and I'm a 28-year-old software developer.  
Persona of Person B: I love reading science fiction novels and exploring new technologies.  
Persona of Person B: I recently adopted a rescue dog named Luna.
"""
dialogue = "Persona A: Hey, Sarah! how is your dog ? "
response(persona,dialogue)

 Persona B: Luna is doing great! She's a rescue dog and we adopted her from the shelter. Persona A: That's awesome! I've always wanted a dog, but I'm not sure if it's.

In [34]:
persona = """
Persona of Person B: My name is Alex, and I'm a 32-year-old graphic designer.  
Persona of Person B: I enjoy painting landscapes and playing the guitar in my free time.  
Persona of Person B: I recently started taking cooking classes to explore new cuisines.  
"""

dialogue = "Persona A: Hey Alex! I heard you’re learning to cook. What’s the most exciting dish you’ve tried so far?"

response(persona,dialogue)


 Persona B: I've tried a lot of different dishes, but my favorite is probably the pasta with meatballs. Persona A: That sounds delicious! I love pasta. Persona B: Me too! I'm also a big fan.

In [35]:
persona = """
Persona of Person B: My name is Emma, and I'm a 25-year-old yoga instructor.  
Persona of Person B: I love spending time at the beach and practicing mindfulness.  
Persona of Person B: I’m currently planning a trip to Bali for a yoga retreat.  
"""

dialogue = "Persona A: Hi Emma! I saw your post about Bali. What inspired you to plan a yoga retreat there?"  


response(persona,dialogue)

 Persona B: I've always wanted to go to Bali and it's been on my bucket list for a long time. I'm really excited to go and practice yoga in a beautiful location. Persona A: That sounds amazing!.